In [16]:
#!pip install keras
#!pip install pandas
#!pip install seaborn

#!pip install tensorflow
#!pip install --ignore-installed --upgrade tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [6]:
df = pd.read_csv('./export_dataframe.csv')
#df

In [7]:
df=df[['Current_Job', 'Current_Company','skills']]
#df

In [10]:
#!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\basha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['skills'] = df['skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['skills'] = df['skills'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [13]:
corpus = list(df['skills'])

In [17]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.text import one_hot

# Declaring vocabulary size as 5000
voc_size = 5000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [18]:
from keras.preprocessing.sequence import pad_sequences 
sent_length = 70
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)

In [20]:
import numpy as np
data = np.arange(0, 2133)
data = pd.Series(data)
df['Title_Encoded']=data
df.head()

,Current_Job,Current_Company,skills,Title_Encoded
0,freelancer,upwork,"management,microsoft excel,custom service,stra...",0
1,project coordinator,devbunch private limited,"microsoft office,custom service,microsoft exce...",1
2,data analyst,daraz,"busi intellig (bi),data analysis,data mining,s...",2
3,corporate marketing manager,no joke marketing,"microsoft office,microsoft excel,powerpoint,re...",3
4,software engineer,keeptruckin,"java,rubi rails,problem solving,android develo...",4


In [21]:
y = df['Title_Encoded']
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=30)

In [24]:
X_train = X_final
y_train = y_final

In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras.models import Sequential
from keras import layers
from keras import regularizers
embedding_vector_features=70
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(layers.Dense(64, activation='relu'))
model.add(Dense(2133,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 70, 70)            350000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               68400     
                                                                 
 dense_2 (Dense)             (None, 64)                6464      
                                                                 
 dense_3 (Dense)             (None, 2133)              138645    
                                                                 
Total params: 563,509
Trainable params: 563,509
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
y_train_oh = to_categorical(y_train,num_classes=2133)
y_test_oh = to_categorical(y_test,num_classes=2133)

In [30]:
### Finally Training
model.fit(X_train,y_train_oh,validation_data=(X_test,y_test_oh),epochs=40,batch_size=32)

Epoch 1/40
67/67 [==============================] - 8s 76ms/step - loss: 7.6774 - accuracy: 0.0000e+00 - val_loss: 7.6658 - val_accuracy: 0.0014
Epoch 2/40
67/67 [==============================] - 5s 69ms/step - loss: 7.6719 - accuracy: 0.0000e+00 - val_loss: 7.6507 - val_accuracy: 0.0043
Epoch 3/40
67/67 [==============================] - 5s 68ms/step - loss: 7.6396 - accuracy: 0.0000e+00 - val_loss: 7.5865 - val_accuracy: 0.0099
Epoch 4/40
67/67 [==============================] - 6s 82ms/step - loss: 7.4517 - accuracy: 4.6882e-04 - val_loss: 7.2479 - val_accuracy: 0.0085
Epoch 5/40
67/67 [==============================] - 6s 87ms/step - loss: 7.0923 - accuracy: 4.6882e-04 - val_loss: 6.7525 - val_accuracy: 0.0071
Epoch 6/40
67/67 [==============================] - 6s 84ms/step - loss: 6.6236 - accuracy: 9.3765e-04 - val_loss: 6.1896 - val_accuracy: 0.0227
Epoch 7/40
67/67 [==============================] - 5s 79ms/step - loss: 6.0893 - accuracy: 0.0056 - val_loss: 5.5466 - val_accura

In [31]:
def pred_result(data,model):

    # Perform cleaning of input data before prediction
    # Remove Stop Word
    
    data = data.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Lemmatization
    data = data.apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    data_list = list(data)
    voc_size = 5000
    one_hot_repr = [one_hot(words,voc_size) for words in data_list]
    sent_length = 70
    embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)    
    X_pred=np.array(embedded_docs)
    model.fit(X_train,y_train_oh,epochs=40,batch_size=32, verbose=0)
    results = model.predict(X_pred)
    return results

In [32]:
result = pred_result(df['skills'],model)


In [33]:
pred_Job_title = []
pred_Company = []
for i in range(len(result)):
    pred_Job_title.append(df.iloc[i,0])
    pred_Company.append(df.iloc[i,1])

In [34]:
predicted_Title = pd.Series(pred_Job_title)
predicted_Company = pd.Series(pred_Company)

data = {'Skills':  list(df['skills']),
        'Predicted_Title': predicted_Title,
        'Predicted_Company': predicted_Company 
        }

df_predicted = pd.DataFrame (data, columns = ['Skills','Predicted_Title','Predicted_Company'])



df_predicted.head()

,Skills,Predicted_Title,Predicted_Company
0,"management,microsoft excel,custom service,stra...",freelancer,upwork
1,"microsoft office,custom service,microsoft exce...",project coordinator,devbunch private limited
2,"busi intellig (bi),data analysis,data mining,s...",data analyst,daraz
3,"microsoft office,microsoft excel,powerpoint,re...",corporate marketing manager,no joke marketing
4,"java,rubi rails,problem solving,android develo...",software engineer,keeptruckin


In [36]:
df_predicted.to_csv('./Prediction_of_skills_and_job.csv')